# Dataset Preparation
- Inputs:
    - American Housing Survey (AHS) for survey years 2015, 2017, 2019, 2021, 2023.
    - HUD Income Limits for Select Metro Areas and Years (2015, 2017, 2019, 2021, 2023)
        - 12060: Atlanta-Sandy Springs-Roswell, GA
        - 14460: Boston-Cambridge-Newton, MA-NH
        - 16980: Chicago-Naperville-Elgin, IL-IN-WI
        - 19100: Dallas-Fort Worth-Arlington, TX
        - 19820: Detroit-Warren-Dearborn, MI
        - 26420: Houston-The Woodlands-Sugar Land, TX
        - 31080: Los Angeles-Long Beach-Anaheim, CA
        - 33100: Miami-Fort Lauderdale-West Palm Beach, FL
        - 35620: New York-Newark-Jersey City, NY-NJ-PA
        - 37980: Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
        - 38060: Phoenix-Mesa-Scottsdale, AZ
        - 40140: Riverside-San Bernardino-Ontario, CA
        - 41860: San Francisco-Oakland-Hayward, CA
        - 42660: Seattle-Tacoma-Bellevue, WA
        - 47900: Washington-Arlington-Alexandria, DC-VA-MD-WV
- Output:
    - A panel dataset (2015 - 2023) that includes an `'AMI'` variable that categorizes each observation/household (identified by the `'CONTROL'` variable) by Area Median Income level based on the corresponding Income Limit thresholds for that Metropolitan and fiscal year defined by the U.S. Department of Housing and Urban Development (HUD). 

This notebook accomplishes the following parts of the workflow/analysis:
1. Complies a longitudinal dataset.
2. Isolates the AHS data to renter and owner-occupied housing units.
3. Identfies unique renter/owner occupied housing units by AMI level for the 15 largest metropolitan areas identified in the AHS:
    - Above LI: >80% AMI
    - Low-Income (LI): <= 80% AMI
    - Very Low-Income (VLI): <= 50% AMI
    - Extemely Low-Income (ELI): <= 30% AMI

In [1]:
#Importing Libraries
import requests
from io import StringIO

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

## Data Access
The AHS Public Use Files (PUFs) can be downloaded from census.gov:

- [2015 AHS National PUF](https://www.census.gov/programs-surveys/ahs/data/2015/ahs-2015-public-use-file--puf-/ahs-2015-national-public-use-file--puf-.html)
- [2017 AHS National PUF](https://www.census.gov/programs-surveys/ahs/data/2017/ahs-2017-public-use-file--puf-/ahs-2017-national-public-use-file--puf-.html)
- [2019 AHS National PUF](https://www.census.gov/programs-surveys/ahs/data/2019/ahs-2019-public-use-file--puf-/ahs-2019-national-public-use-file--puf-.html)
- [2021 AHS National PUF](https://www.census.gov/programs-surveys/ahs/data/2021/ahs-2021-public-use-file--puf-/ahs-2021-national-public-use-file--puf-.html)
- [2023 AHS National PUF](https://www.census.gov/programs-surveys/ahs/data/2023/ahs-2023-public-use-file--puf-/ahs-2023-national-public-use-file--puf-.html)

Note: This analysis uses the PUF "Flat File."

In [2]:
#Selected relevant variables for the analysis
columns_15 = ['CONTROL', 'WEIGHT', 'OMB13CBSA', 'VACANCY', 'VACMONTHS', 'TENURE', 'NUMPEOPLE', 'HINCP', 'RENT', 'TOTHCAMT',
           'HUDSUB', 'RENTSUB', 'RENTCNTRL', 'BLD', 'YRBUILT', 'BEDROOMS', 'HHMOVE']

columns = ['CONTROL', 'WEIGHT', 'OMB13CBSA', 'VACANCY', 'VACMONTHS', 'TENURE', 'NUMPEOPLE', 'HINCP', 'RENT', 'TOTHCAMT',
           'HUDSUB', 'RENTSUB', 'RENTCNTRL', 'BLD', 'YRBUILT', 'BEDROOMS', 'SAMEHHLD', 'HHMOVE']

#Loading 2015 AHS METRO PUF w/ selected columns
df_15 = pd.read_csv('data/ahs/ahs2015n.csv',
                    usecols=columns_15)

#Loading 2017 AHS METRO PUF w/ selected columns
df_17 = pd.read_csv('data/ahs/ahs2017n.csv',
                    usecols=columns)

#Loading 2019 AHS METRO PUF w/ selected columns
df_19 = pd.read_csv('data/ahs/ahs2019n.csv',
                    usecols=columns)
#Loading 2021 AHS METRO PUF w/ selected columns
df_21 = pd.read_csv('data/ahs/ahs2021n.csv',
                    usecols=columns)

#Loading 2023 AHS METRO PUF w/ selected columns
df_23 = pd.read_csv('data/ahs/ahs2023n.csv',
                    usecols=columns)

In [3]:
#Storing the DataFrames in a dictionary with full survey years as keys
dfs_by_year = {
    2023: df_23,
    2021: df_21,
    2019: df_19,
    2017: df_17,
    2015: df_15
}

#Previewing each df
for year, df in dfs_by_year.items():
    print(f'--- AHS {year} ---')
    print(df.head(), '\n')

--- AHS 2023 ---
      CONTROL  RENT TENURE RENTCNTRL RENTSUB OMB13CBSA       WEIGHT   BLD  \
0  '11000002'  1600   '-6'      '-6'     '8'   '99998'   813.890194  '03'   
1  '11000003'   840    '2'      '-6'     '8'   '99998'   581.103231  '03'   
2  '11000005'    -6    '1'      '-6'    '-6'   '99998'  7335.965001  '02'   
3  '11000006'    -6    '1'      '-6'    '-6'   '99998'  6562.865941  '02'   
4  '11000008'   800    '2'      '-6'     '8'   '99998'  1490.800600  '06'   

   HHMOVE  NUMPEOPLE SAMEHHLD  YRBUILT  BEDROOMS  VACMONTHS HUDSUB VACANCY  \
0      -6         -6     '-6'     1980         4          0   '-6'    '02'   
1    2023          3      '4'     1970         2         -6    '3'    '-6'   
2    1995          2      '1'     1970         4         -6   '-6'    '-6'   
3    2019          3      '4'     1970         3         -6   '-6'    '-6'   
4    2019          1      '4'     1920         1         -6    '3'    '-6'   

    HINCP  TOTHCAMT  
0      -6        -6  
1   480

In [4]:
#Checking the shape/info of each DataFrame
for year, df in dfs_by_year.items():
    print(f'\n--- AHS {year} ---')
    print('Shape:', df.shape)
    df.info()


--- AHS 2023 ---
Shape: (55669, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55669 entries, 0 to 55668
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CONTROL    55669 non-null  object 
 1   RENT       55669 non-null  int64  
 2   TENURE     55669 non-null  object 
 3   RENTCNTRL  55669 non-null  object 
 4   RENTSUB    55669 non-null  object 
 5   OMB13CBSA  55669 non-null  object 
 6   WEIGHT     55669 non-null  float64
 7   BLD        55669 non-null  object 
 8   HHMOVE     55669 non-null  int64  
 9   NUMPEOPLE  55669 non-null  int64  
 10  SAMEHHLD   55669 non-null  object 
 11  YRBUILT    55669 non-null  int64  
 12  BEDROOMS   55669 non-null  int64  
 13  VACMONTHS  55669 non-null  int64  
 14  HUDSUB     55669 non-null  object 
 15  VACANCY    55669 non-null  object 
 16  HINCP      55669 non-null  int64  
 17  TOTHCAMT   55669 non-null  int64  
dtypes: float64(1), int64(8), object(9)
memory usage: 

## Setting up and Preparing DataFrames for Longitudinal Analysis
- Adding `SRVYEAR` variable/column to identify the year of the AHS dataset that will be concatenated later in the analysis
- Using `.concat()` to stack the DataFrame vertically (long format)
- Stripping the extra `''` from the sting values 

In [5]:
#Add the survey year as a new column in each DataFrame
dfs_by_year = {
    year: df.assign(SRVYEAR=int(year)) for year, df in dfs_by_year.items()
}

In [6]:
#Checking the DataFrames to see if the column was added
for year, df in dfs_by_year.items():
    print(f'--- AHS {year} ---')
    print(df.head(), '\n')

--- AHS 2023 ---
      CONTROL  RENT TENURE RENTCNTRL RENTSUB OMB13CBSA       WEIGHT   BLD  \
0  '11000002'  1600   '-6'      '-6'     '8'   '99998'   813.890194  '03'   
1  '11000003'   840    '2'      '-6'     '8'   '99998'   581.103231  '03'   
2  '11000005'    -6    '1'      '-6'    '-6'   '99998'  7335.965001  '02'   
3  '11000006'    -6    '1'      '-6'    '-6'   '99998'  6562.865941  '02'   
4  '11000008'   800    '2'      '-6'     '8'   '99998'  1490.800600  '06'   

   HHMOVE  NUMPEOPLE SAMEHHLD  YRBUILT  BEDROOMS  VACMONTHS HUDSUB VACANCY  \
0      -6         -6     '-6'     1980         4          0   '-6'    '02'   
1    2023          3      '4'     1970         2         -6    '3'    '-6'   
2    1995          2      '1'     1970         4         -6   '-6'    '-6'   
3    2019          3      '4'     1970         3         -6   '-6'    '-6'   
4    2019          1      '4'     1920         1         -6    '3'    '-6'   

    HINCP  TOTHCAMT  SRVYEAR  
0      -6        -6 

In [7]:
#checking the average of the `WEIGHT` variables. Each year should average ~2,000
avg23 = np.average(df_23['WEIGHT'])
print(avg23)

2610.8720472795485


### Concatenating the 2015, 2017, 2019, 2021, and 2023 AHS PUFs to produce a **long-format panel dataset**.
This will allow:
- **Track units over time** --> This requires **multiple rows per unit** (i.e., one row per unit * per year).
- **Identify the same unit** --> This is already possible with the `CONTROL` variable.
- **Compare status across years** (e.g., 2015 vs. 2023) --> This is easiest when **filtering by year** can be cleanly, which is what the `SRVYEAR` column enables.

In [8]:
#Stack all years into a single long-form DataFrame
ahs_long_df = pd.concat(dfs_by_year.values(), ignore_index=True)

In [9]:
#Inspecting the shape and preview the concatenated DataFrames
print(ahs_long_df.shape)
print(ahs_long_df.info())
print(ahs_long_df.head())
print(ahs_long_df['SRVYEAR'].value_counts())

(319240, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319240 entries, 0 to 319239
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   CONTROL    319240 non-null  object 
 1   RENT       319240 non-null  int64  
 2   TENURE     319240 non-null  object 
 3   RENTCNTRL  319240 non-null  object 
 4   RENTSUB    319240 non-null  object 
 5   OMB13CBSA  319240 non-null  object 
 6   WEIGHT     319240 non-null  float64
 7   BLD        319240 non-null  object 
 8   HHMOVE     319240 non-null  int64  
 9   NUMPEOPLE  319240 non-null  int64  
 10  SAMEHHLD   249747 non-null  object 
 11  YRBUILT    319240 non-null  int64  
 12  BEDROOMS   319240 non-null  int64  
 13  VACMONTHS  319240 non-null  int64  
 14  HUDSUB     319240 non-null  object 
 15  VACANCY    319240 non-null  object 
 16  HINCP      319240 non-null  int64  
 17  TOTHCAMT   319240 non-null  int64  
 18  SRVYEAR    319240 non-null  int64  
dtypes: float64

In [10]:
#Checking for unique unit-year combinations, the lines of code should produce the same output
print(len(ahs_long_df))
print(ahs_long_df[['CONTROL', 'SRVYEAR']].drop_duplicates().shape[0])

319240
319240


In [11]:
#Ensuring no columns were dropped
print(ahs_long_df.columns)

Index(['CONTROL', 'RENT', 'TENURE', 'RENTCNTRL', 'RENTSUB', 'OMB13CBSA',
       'WEIGHT', 'BLD', 'HHMOVE', 'NUMPEOPLE', 'SAMEHHLD', 'YRBUILT',
       'BEDROOMS', 'VACMONTHS', 'HUDSUB', 'VACANCY', 'HINCP', 'TOTHCAMT',
       'SRVYEAR'],
      dtype='object')


In [12]:
#Looking for missing or duplicated `CONTROL`-year rows
dupes = ahs_long_df.duplicated(subset=["CONTROL", "SRVYEAR"], keep=False)
print("Duplicated CONTROL + SRVYEAR rows:", dupes.sum())

Duplicated CONTROL + SRVYEAR rows: 0


In [13]:
#Checking counts by year to confirm stacking
ahs_long_df["SRVYEAR"].value_counts().sort_index()

SRVYEAR
2015    69493
2017    66752
2019    63185
2021    64141
2023    55669
Name: count, dtype: int64

##### Stripping the extra quotation marks `''` from the sting values of the entire dataset to make coding a cleaner/easier

In [14]:
ahs_long_df[ahs_long_df.select_dtypes(include='object')
            .columns] = ahs_long_df.select_dtypes(include='object').apply(lambda col: col.str.strip("'"))
ahs_long_df.head()

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,SAMEHHLD,YRBUILT,BEDROOMS,VACMONTHS,HUDSUB,VACANCY,HINCP,TOTHCAMT,SRVYEAR
0,11000002,1600,-6,-6,8,99998,813.890194,03,-6,-6,-6,1980,4,0,-6,02,-6,-6,2023
1,11000003,840,2,-6,8,99998,581.103231,03,2023,3,4,1970,2,-6,3,-6,48000,1093,2023
2,11000005,-6,1,-6,-6,99998,7335.965001,02,1995,2,1,1970,4,-6,-6,-6,292500,810,2023
3,11000006,-6,1,-6,-6,99998,6562.865941,02,2019,3,4,1970,3,-6,-6,-6,56000,489,2023
4,11000008,800,2,-6,8,99998,1490.800600,06,2019,1,4,1920,1,-6,3,-6,36000,845,2023


### Creating new DataFrame to exclude "Units occupied without payment of rent"
- The new `ahs_panel_df` DataFrame will include owner-occupied units (`'1'`), renter--occupied units (`'2'`), and not applicable/vacant units (`'-6'`).

In [15]:
#Examining `TENURE` variable
ahs_long_df['TENURE'].value_counts()

TENURE
1     162666
2     111084
-6     41729
3       3761
Name: count, dtype: int64

The new DataFrame will consist of:
- Owner-occupied housing units (`'1'`): 162,666
- Renter-occupied housing units (`'2'`): 111,084
- Not Applicable (`'-6'`): 41,729 -- _need this for vacant unit analysis_

The new DataFrame will disregard:
- Units occupied w/o payment of rent (`'3'`): 3,761

In [16]:
#Looking at the characteristic of 'HINCP' to see if "N" values exists in the DataFrame or if its just '-6' values
print(ahs_long_df['HINCP'].dtype)
print(ahs_long_df['HINCP'].unique())

#Number of unique values in the df
print(ahs_long_df['HINCP'].nunique())

int64
[    -6  48000 292500 ...    364 327100 111660]
16891


In [17]:
#Looking at the number of renter & owner occupied housing units that have "not applicable" ('-6') values for household income ('HINCP')
ahs_long_df[(ahs_long_df['TENURE'] == '1') | (ahs_long_df['TENURE'] == '2')].groupby('HINCP').size().get(-6,0)

0

In [18]:
ahs_long_df[((ahs_long_df['TENURE'] == '1') | (ahs_long_df['TENURE'] == '2')) & (ahs_long_df['HINCP'] == -6)].shape[0]

0

In [19]:
#Checking any NA/-6 values among renter houhseholds in the df
grouped = ahs_long_df[(ahs_long_df['TENURE'] == '1') | (ahs_long_df['TENURE'] == '2')].groupby('HINCP').size()

#Check if -6 is in the index before using .loc
if -6 in grouped.index:
    count_minus_6 = grouped.loc[-6]
else:
    count_minus_6 = 0

print("Number of owner/renter-occupied housing units with HINCP == -6:", count_minus_6)

Number of owner/renter-occupied housing units with HINCP == -6: 0


All owner/renter observations have a dollar amount within the `'HINCP'` variable.

In [20]:
#Looking at sum of renter/owner-occupied/vacant housing units
rent_own_count = ((ahs_long_df['TENURE'] == '1') | (ahs_long_df['TENURE'] == '2') | (ahs_long_df['TENURE'] == '-6')).sum()
print(rent_own_count)

315479


In [21]:
#New df consisting of owner/renter-occupied + vacant housing units
ahs_panel_df = ahs_long_df[ahs_long_df['TENURE'] != '3']

#Checking if `ahs_panel_df` was filtered correctly
len(ahs_panel_df)

315479

Owner-occupied housing units ('1'): 162,666 + Renter-occupied housing units ('2'): 111,084 + Not Applicable/Vacant ('6'): 41,729 = 315,479

### Examinning the household income `'HINCP'` and number of people in the unit `'NUMPEOPLE'` variables for renter/owner-occupied housing units
- These variables are critical for matching observations in the AHS to the correct Area Median Income (AMI) category based on HUD's Income Limit thresholds.

In [22]:
pd.options.display.float_format = '{:.2f}'.format #changing numeric format to see the dollar amount more clearly

ahs_panel_df.groupby('TENURE')[['HINCP', 'NUMPEOPLE']].describe()

HINCP                                                            \
           count      mean       std       min      25%      50%       75%   
TENURE                                                                       
-6      41729.00     -6.00      0.00     -6.00    -6.00    -6.00     -6.00   
1      162666.00 110403.59 138828.23  -5000.00 40000.00 78000.00 135000.00   
2      111084.00  52883.59  77385.25 -10000.00 14100.00 34200.00  68000.00   

                  NUMPEOPLE                                           
              max     count  mean  std   min   25%   50%   75%   max  
TENURE                                                                
-6          -6.00  41729.00 -6.00 0.00 -6.00 -6.00 -6.00 -6.00 -6.00  
1      5786000.00 162666.00  2.58 1.43  1.00  2.00  2.00  3.00 18.00  
2      6445000.00 111084.00  2.27 1.48  1.00  1.00  2.00  3.00 19.00

In [23]:
#Examining the the oultiers
ahs_panel_df.groupby('TENURE')[['HINCP', 'NUMPEOPLE']].quantile([0, .01, .2,.4,.6,.8,.99])

HINCP  NUMPEOPLE
TENURE                          
-6     0.00     -6.00      -6.00
       0.01     -6.00      -6.00
       0.20     -6.00      -6.00
       0.40     -6.00      -6.00
       0.60     -6.00      -6.00
       0.80     -6.00      -6.00
       0.99     -6.00      -6.00
1      0.00  -5000.00       1.00
       0.01      0.00       1.00
       0.20  32300.00       1.00
       0.40  61200.00       2.00
       0.60  97000.00       3.00
       0.80 153000.00       4.00
       0.99 592415.00       7.00
2      0.00 -10000.00       1.00
       0.01      0.00       1.00
       0.20  11400.00       1.00
       0.40  25000.00       1.00
       0.60  45000.00       2.00
       0.80  79800.00       3.00
       0.99 315000.00       7.00

In [24]:
#Looking at the number of renter/owner-occpuied houing units with negative incomes
ahs_panel_df.loc[ahs_panel_df["HINCP"] < 0].groupby('TENURE')[['HINCP', 'NUMPEOPLE']].describe()

HINCP                                                                \
          count     mean     std       min      25%      50%      75%     max   
TENURE                                                                          
-6     41729.00    -6.00    0.00     -6.00    -6.00    -6.00    -6.00   -6.00   
1         10.00 -2592.40 1887.42  -5000.00 -4440.00 -2450.00 -1175.00   -4.00   
2          5.00 -5160.00 3260.06 -10000.00 -5000.00 -5000.00 -5000.00 -800.00   

       NUMPEOPLE                                           
           count  mean  std   min   25%   50%   75%   max  
TENURE                                                     
-6      41729.00 -6.00 0.00 -6.00 -6.00 -6.00 -6.00 -6.00  
1          10.00  2.90 3.63  1.00  1.00  2.00  2.75 13.00  
2           5.00  1.80 1.10  1.00  1.00  1.00  3.00  3.00

In [25]:
ahs_panel_df.loc[ahs_panel_df["HINCP"] < 0, 'TENURE'].value_counts()

TENURE
-6    41729
1        10
2         5
Name: count, dtype: int64

There are 15 observations in the sample that have negative values for `'HINCP'`. These observations will not be removed from the DataFrame since it is possible (or at least, for the American Community Survey it is, that these observations are meaningful if a renter/owner household moves into a unit with no income. Plus, the AHS tracks _**housing units**_(`'CONTROL'`), not households.   

In addition, housing unit occupants are catagorized by Area Median Income (AMI) later in this data preparation notebook, these observations would be catagorized as Extremely Low-Income with no major effect to the data and the panel dataset will have less "gaps" in observations.

## Loading in Income Limits to catagorize housing units by Area Median Income (AMI) defined by the U.S. Department of Housing and Urban Development (HUD) Income Limits

Accessing 2017, 2019, 2021, and 2023 Income Limit data via the API Function from HUD USER (https://www.huduser.gov/portal/dataset/fmr-api.h) and will merge the newly created `staked_il_df` on the CBSA code (`'OMB13CBSA'`) variable to the AHS data and then create a new `'AMI'` categorical variable with the following values:
- Above LI: Above Low-Income
    - Households above the low-income thresholds of 80% Area Median Income (AMI).
    - Households above 80% AMI are not elibigle for federal rental subsidies in most cases.
- LI: Low-Income
    - Renter households with incomes at or below 80% AMI.
- VLI: Very Low-Income
    - Renter Households with incomes at or below 50% AMI.
- ELI: Extremely Low-Income
    - Renter Households with incomes at or below 30% AMI

**NOTE:** HUD _**does not**_ support API requests for years prior to 2017. I will need to download the FY2015 from HUDUSER, isolate the corresponding CBSA/Metro Area, and merge that into the API-requested DataFrame.

A stacked DataFrame will be created with a new column `'il_fiscal_year'` to merge the stacked income limits DataFrame with the longitudinal AHS data to match an household's `'HINCP'` (household income) with the corresponding `'AMI'` category based on the income limits of that fiscal year,

The AHS National PUF only surveys the top 15 largest metropolitan areas. Due to this limitation, the following code will load in HUD's Income Limits data for the following metropolitan areas. AHS observations that are not located within the top 15 largest metropolitan areas (`'99998'` - All other metroplitan areas not in Top 15, and `'99999'` - Not in a metropolitan area) will not have corresponding income limit data once the two datasets are merged amd will be omitted from AMI classification.
- Atlanta-Sandy Springs-Roswell, GA (12060)
- Chicago-Naperville-Elgin, IL-IN-WI (16980)
- Dallas-Fort Worth-Arlington, TX (19100)
- Detroit-Warren-Dearborn, MI (19820)
- Houston-The Woodlands-Sugar Land, TX (26420)
- Philadelphia-Camden-Wilmington, PA-NJ-DE-MD (37980)
- Phoenix-Mesa-Scottsdale, AZ (38060)
- Riverside-San Bernardino-Ontario, CA (40140)
- Washington-Arlington-Alexandria, DC-VA-MD (47900)

**Note:** HUD uses a different geographical definition (`'hud_area_code'`) to establish Income Limits for the metropolitan areas below. HUD Income Limts use "HUD Metro FMR Area" which are different than the AHS geographical definition (`'OMB13CBSA'`) for that area. For the purposes of this analysis, the "central" county of the HUD Metro FMR Area within the AHS defined metropolitan area will be used for the purposes of assigning an AMI catagory for an observation. For example, the AHS-defined "Miami-Fort Lauderdale-West Palm Beach, FL" metropolitan area will use the "Miami-Miami Beach-Kendall, FL HUD Metro FMR Area" since Miami-Dade county uses that FMR area; even though Fort Lauderdale and West Palm Beach are part of the AHS-defined metropolitan area but have different income thresholds for the purpose of establishing HUD Income Limts.
- Boston-Cambridge-Quincy, MA-NH (14460) - Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area (Suffolk County)
- Miami-Fort Lauderdale-West Palm Beach, FL (33100) - Miami-Miami Beach-Kendall, FL HUD Metro FMR Area (Miami-Dade County)
- Los Angeles-Long Beach-Anaheim, CA (31080) - Los Angeles-Long Beach-Glendale, CA HUD Metro FMR Area (Los Angeles County)
- New York-Newark-Jersey City, NY-NJ-PA (35620) - New York, NY HUD Metro FMR Area (New York County)
- San Francisco-Oakland-Hayward, CA (41860) - San Francisco, CA HUD Metro FMR Area (San Francisco County)
- Seattle-Tacoma-Bellevue, WA (42660) - Seattle-Bellevue, WA HUD Metro FMR Area (King County)

In [26]:
#API key
API_KEY = 'API_KEY'

#Identifying metro area
cbsa_codes = [
    'METRO12060M12060', #ATL
    'METRO14460MM1120', #BOS
    'METRO16980M16980', #CHI
    'METRO19100M19100', #DAL
    'METRO19820M19820', #DET
    'METRO26420M26420', #HOU
    'METRO31080MM4480', #LA
    'METRO33100MM5000', #MIA
    'METRO35620MM5600', #NYC
    'METRO37980M37980', #PHI
    'METRO38060M38060', #PHX
    'METRO40140M40140', #RIVERSIDE, CA
    'METRO41860MM7360', #SF
    'METRO42660MM7600', #SEA
    'METRO47900M47900'  #DC
]

#Identifying years for the staked DataFrame
years = [2017, 2019, 2021, 2023] 

#Initialize both dictionaries & storage
metro_dict = {}  
year_dict = {}   
all_rows = []

#API request & loop through CBSA and year
for cbsa_code in cbsa_codes:
    for year in years:
        url = f'https://www.huduser.gov/hudapi/public/il/data/{cbsa_code}?year={year}'
        headers = {'Authorization': f'Bearer {API_KEY}'}
        response = requests.get(url, headers=headers)
    
        json_data = response.json()
        data = json_data.get('data',{})
        
        #Structuring the json into a panda dataframe that is reflective of the excel file avaialbe on the web
        #Parse row
        row = {
            'hud_area_code': cbsa_code,
            'hud_area_name': data.get('area_name'),
            'il_fiscal_year': data.get('year'),
            'median_income': data.get('median_income')
        }
        
        #Add l50 (Very Low), ELI (Exrememly Low), l80 (Low)
        for i in range(1, 9):
            row[f'l50_{i}'] = data.get('very_low', {}).get(f'il50_p{i}')
            row[f'ELI_{i}'] = data.get('extremely_low', {}).get(f'il30_p{i}')
            row[f'l80_{i}'] = data.get('low', {}).get(f'il80_p{i}')
            
        #Add dictionaries
        metro_dict.setdefault(cbsa_code, []).append(row)
        year_dict.setdefault(int(data.get('year')), []).append(row)
        
        #Adding to masterlist
        all_rows.append(row)
        
#Creating the final Dataframe for income limits from 2017-2023 for the target metro areas
metro_staked_il_df = pd.DataFrame(all_rows)
metro_staked_il_df

,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,ELI_1,l80_1,l50_2,ELI_2,l80_2,...,l80_5,l50_6,ELI_6,l80_6,l50_7,ELI_7,l80_7,l50_8,ELI_8,l80_8
0,METRO12060M12060,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",2017,69700,24400,14650,39050,27900,16750,44600,...,60250,40450,32960,64700,43250,37140,69150,46050,41320,73600
1,METRO12060M12060,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",2019,79700,27900,16750,44650,31900,19150,51000,...,68850,46250,34590,73950,49450,39010,79050,52650,43430,84150
2,METRO12060M12060,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",2021,86200,30200,18100,48300,34500,20700,55200,...,74500,50000,35580,80000,53450,40120,85500,56900,44660,91050
3,METRO12060M12060,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",2023,103500,35750,21500,57200,40850,24550,65350,...,88200,59250,40280,94750,63350,45420,101250,67400,50560,107800
4,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2017,103400,36200,21700,54750,41400,24800,62550,...,84450,60000,36000,90700,64150,38450,96950,68250,41320,103200
5,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2019,113300,41500,24900,62450,47400,28450,71400,...,96350,68750,41250,103500,73500,44100,110650,78250,46950,117750
6,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2021,120800,47000,28200,70750,53700,32200,80850,...,109150,77850,46700,117250,83250,49950,125350,88600,53150,133400
7,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2023,149300,51950,31150,82950,59400,35600,94800,...,127950,86100,51650,137450,92050,55200,146900,97950,58750,156400
8,METRO16980M16980,"Chicago-Joliet-Naperville, IL HUD Metro FMR Area",2017,79000,27650,16600,44250,31600,19000,50600,...,68300,45850,32960,73350,49000,37140,78400,52150,41320,83450
9,METRO16980M16980,"Chicago-Joliet-Naperville, IL HUD Metro FMR Area",2019,89100,31200,18750,49950,35650,21400,57050,...,77050,51700,34590,82750,55250,39010,88450,58850,43430,94150


In [27]:
#Loading in HUD's 2015 Income Limit Data for target CBSAs

#File path
file_path = 'data/ahs/Section8_Rev.xlsx'

#Using excel rows to identify target CBSAs (1-based indexing)
target_rows_excel = [
    609, #ATL
    2182, #BOS 
    773, #CHI
    3880, #DAL
    2327, #DET
    3924, #HOU
    206, #LA
    525, #MIA
    3123, #NYC
    3561, #PHI
    105, #PHX
    220, #RIVERSIDE, CA
    225, #SF
    4513, #SEA
    482 #DC
]

#Convert to 0-based pandas row indices
target_indices = [i - 1 for i in target_rows_excel]

#Read the file, only desired rows (skip all others)
il_metro_15_df = pd.read_excel(
    file_path,
    header=None,
    skiprows=lambda x: x not in target_indices
)

#Read only the header row separately (row 0) for column names
headers = pd.read_excel(file_path, nrows=0).columns.tolist()

#Assign headers back into DataFrame
il_metro_15_df.columns = headers
il_metro_15_df

,State_Alpha,fips2000,State,County,County_Name,CBSASub,Metro_Area_Name,fips2010,median2015,l50_1,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,MSA,county_town_name,state_name,metro
0,AZ,401399999,4,13,Maricopa County,METRO38060M38060,"Phoenix-Mesa-Glendale, AZ MSA",401399999,64000,22400,...,46100,51200,55300,59400,63500,67600,6200,Maricopa County,Arizona,1
1,CA,603799999,6,37,Los Angeles County,METRO31100MM4480,"Los Angeles-Long Beach, CA HUD Metro FMR Area",603799999,63000,29050,...,59800,66400,71750,77050,82350,87650,4480,Los Angeles County,California,1
2,CA,606599999,6,65,Riverside County,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",606599999,60500,21750,...,44750,49700,53700,57700,61650,65650,6780,Riverside County,California,1
3,CA,607599999,6,75,San Francisco County,METRO41860MM7360,"San Francisco, CA HUD Metro FMR Area",607599999,101900,41050,...,84500,93850,101400,108900,116400,123900,7360,San Francisco County,California,1
4,DC,1100199999,11,1,District of Columbia,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",1100199999,109200,38250,...,61200,68000,73450,78900,84350,89800,8840,District of Columbia,District of Columbia,1
5,FL,1208699999,12,86,Miami-Dade County,METRO33100MM5000,"Miami-Miami Beach-Kendall, FL HUD Metro FMR Area",1208699999,49900,23700,...,48750,54150,58500,62850,67150,71500,5000,Miami-Dade County,Florida,1
6,GA,1312199999,13,121,Fulton County,METRO12060M12060,"Atlanta-Sandy Springs-Marietta, GA HUD Metro F...",1312199999,68300,23900,...,49100,54550,58950,63300,67650,72050,520,Fulton County,Georgia,1
7,IL,1703199999,17,31,Cook County,METRO16980M16980,"Chicago-Joliet-Naperville, IL HUD Metro FMR Area",1703199999,76000,26600,...,54750,60800,65700,70550,75400,80300,1600,Cook County,Illinois,1
8,MA,2502507000,25,25,Suffolk County,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2502507000,98500,34500,...,62750,69700,75300,80900,86450,92050,1120,Boston city,Massachusetts,1
9,MI,2616399999,26,163,Wayne County,METRO19820M19820,"Detroit-Warren-Livonia, MI HUD Metro FMR Area",2616399999,67700,23700,...,48750,54150,58500,62850,67150,71500,2160,Wayne County,Michigan,1


#### Structuring the FY2015 Income Limit DataFrame to match the API-request DataFrame above.

In [28]:
#Dropping columns that are not relevant 
il_metro_15_df.drop(columns=['State_Alpha', 'fips2000', 'State', 'County', 'County_Name', 'fips2010', 'MSA', 'county_town_name',
                           'state_name', 'metro'], inplace=True)

#Adding `year` column
il_metro_15_df['il_fiscal_year'] = 2015

#Replacing the 2015 'hud_area_code' for Los Angeles-Long Beach, CA HUD Metro FMR Area since the CBSA code (31080) is consistent in the AHS but changes in the Income Limits data (2015 = METRO31100MM4480 but '17, '19, '21, '23 = METRO31080MM4480. 
#This will ensure correct merging later in the notebook
il_metro_15_df['CBSASub'] = il_metro_15_df['CBSASub'].replace('METRO31100MM4480', 'METRO31080MM4480')

#Moving `year` to 3rd column (index position 2)
il_metro_15_df = il_metro_15_df[[*il_metro_15_df.columns[:2], 'il_fiscal_year', *il_metro_15_df.columns[2:-1]]]

#Rename columns to match API-requested DataFrame
il_metro_15_df.rename(columns={'CBSASub': 'hud_area_code', 'Metro_Area_Name': 'hud_area_name', 'median2015': 'median_income'}, 
                      inplace=True)
il_metro_15_df

,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,...,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8
0,METRO38060M38060,"Phoenix-Mesa-Glendale, AZ MSA",2015,64000,22400,25600,28800,32000,34600,37150,...,36730,40890,35850,41000,46100,51200,55300,59400,63500,67600
1,METRO31080MM4480,"Los Angeles-Long Beach, CA HUD Metro FMR Area",2015,63000,29050,33200,37350,41500,44850,48150,...,36730,40890,46500,53150,59800,66400,71750,77050,82350,87650
2,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",2015,60500,21750,24850,27950,31050,33550,36050,...,36730,40890,34800,39800,44750,49700,53700,57700,61650,65650
3,METRO41860MM7360,"San Francisco, CA HUD Metro FMR Area",2015,101900,41050,46900,52750,58600,63300,68000,...,43600,46400,65700,75100,84500,93850,101400,108900,116400,123900
4,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",2015,109200,38250,43700,49150,54600,59000,63350,...,40650,43250,47600,54400,61200,68000,73450,78900,84350,89800
5,METRO33100MM5000,"Miami-Miami Beach-Kendall, FL HUD Metro FMR Area",2015,49900,23700,27100,30500,33850,36600,39300,...,36730,40890,37950,43350,48750,54150,58500,62850,67150,71500
6,METRO12060M12060,"Atlanta-Sandy Springs-Marietta, GA HUD Metro F...",2015,68300,23900,27300,30700,34100,36850,39600,...,36730,40890,38200,43650,49100,54550,58950,63300,67650,72050
7,METRO16980M16980,"Chicago-Joliet-Naperville, IL HUD Metro FMR Area",2015,76000,26600,30400,34200,38000,41050,44100,...,36730,40890,42600,48650,54750,60800,65700,70550,75400,80300
8,METRO14460MM1120,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2015,98500,34500,39400,44350,49250,53200,57150,...,36730,40890,48800,55800,62750,69700,75300,80900,86450,92050
9,METRO19820M19820,"Detroit-Warren-Livonia, MI HUD Metro FMR Area",2015,67700,23700,27100,30500,33850,36600,39300,...,36730,40890,37950,43350,48750,54150,58500,62850,67150,71500


#### Stacking the income limit DataFrames

In [29]:
stacked_il_df = pd.concat([il_metro_15_df, metro_staked_il_df], ignore_index=True)

In [30]:
#Exmaining the DataFrame
stacked_il_df

,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,...,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8
0,METRO38060M38060,"Phoenix-Mesa-Glendale, AZ MSA",2015,64000,22400,25600,28800,32000,34600,37150,...,36730,40890,35850,41000,46100,51200,55300,59400,63500,67600
1,METRO31080MM4480,"Los Angeles-Long Beach, CA HUD Metro FMR Area",2015,63000,29050,33200,37350,41500,44850,48150,...,36730,40890,46500,53150,59800,66400,71750,77050,82350,87650
2,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",2015,60500,21750,24850,27950,31050,33550,36050,...,36730,40890,34800,39800,44750,49700,53700,57700,61650,65650
3,METRO41860MM7360,"San Francisco, CA HUD Metro FMR Area",2015,101900,41050,46900,52750,58600,63300,68000,...,43600,46400,65700,75100,84500,93850,101400,108900,116400,123900
4,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",2015,109200,38250,43700,49150,54600,59000,63350,...,40650,43250,47600,54400,61200,68000,73450,78900,84350,89800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,METRO42660MM7600,"Seattle-Bellevue, WA HUD Metro FMR Area",2023,146500,47950,54800,61650,68500,74000,79500,...,51000,54300,70650,80750,90850,100900,109000,117050,125150,133200
71,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",2017,110300,38650,44150,49650,55150,59600,64000,...,41050,43700,52550,60050,67550,75050,81100,87100,93100,99100
72,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",2019,121300,42500,48550,54600,60650,65550,70400,...,45150,48050,54350,62100,69850,77600,83850,90050,96250,102450
73,METRO47900M47900,"Washington-Arlington-Alexandria, DC-VA-MD HUD ...",2021,129000,45150,51600,58050,64500,69700,74850,...,48000,51100,57650,65850,74100,82300,88900,95500,102100,108650


In [31]:
#Inspecting the shape/info of the stacked DataFrame
#Should have 75 rows
print(stacked_il_df.shape)
print(stacked_il_df.info())

(75, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hud_area_code   75 non-null     object
 1   hud_area_name   75 non-null     object
 2   il_fiscal_year  75 non-null     object
 3   median_income   75 non-null     int64 
 4   l50_1           75 non-null     int64 
 5   l50_2           75 non-null     int64 
 6   l50_3           75 non-null     int64 
 7   l50_4           75 non-null     int64 
 8   l50_5           75 non-null     int64 
 9   l50_6           75 non-null     int64 
 10  l50_7           75 non-null     int64 
 11  l50_8           75 non-null     int64 
 12  ELI_1           75 non-null     int64 
 13  ELI_2           75 non-null     int64 
 14  ELI_3           75 non-null     int64 
 15  ELI_4           75 non-null     int64 
 16  ELI_5           75 non-null     int64 
 17  ELI_6           75 non-null     int64 
 18  ELI

Need to convert `'il_fiscal_year'` to and `int64` but need to make sure there aren't any null/NaN values in the column

In [32]:
#checking NaN values in 'il_fiscal_uear' column
stacked_il_df['il_fiscal_year'].isna().sum()

0

In [33]:
#coverting 'il_fiscal_year' into an integer
stacked_il_df['il_fiscal_year'] = stacked_il_df['il_fiscal_year'].astype(int)

In [34]:
print(stacked_il_df.shape)
print(stacked_il_df.info())

(75, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hud_area_code   75 non-null     object
 1   hud_area_name   75 non-null     object
 2   il_fiscal_year  75 non-null     int64 
 3   median_income   75 non-null     int64 
 4   l50_1           75 non-null     int64 
 5   l50_2           75 non-null     int64 
 6   l50_3           75 non-null     int64 
 7   l50_4           75 non-null     int64 
 8   l50_5           75 non-null     int64 
 9   l50_6           75 non-null     int64 
 10  l50_7           75 non-null     int64 
 11  l50_8           75 non-null     int64 
 12  ELI_1           75 non-null     int64 
 13  ELI_2           75 non-null     int64 
 14  ELI_3           75 non-null     int64 
 15  ELI_4           75 non-null     int64 
 16  ELI_5           75 non-null     int64 
 17  ELI_6           75 non-null     int64 
 18  ELI

#### Creating new column `'cbsa_code'` that extracts the 5-digit cbsa code from the `'hud_area_code'` so I can merge it to the AHS `metro_ami_panel_df` DataFrame.

In [35]:
stacked_il_df['OMB13CBSA'] = stacked_il_df['hud_area_code'].str.extract(r'(\d{5})')

#Looking at the DataFrame to see if the Column was created correctly
print(stacked_il_df)

       hud_area_code                                      hud_area_name  \
0   METRO38060M38060                      Phoenix-Mesa-Glendale, AZ MSA   
1   METRO31080MM4480      Los Angeles-Long Beach, CA HUD Metro FMR Area   
2   METRO40140M40140           Riverside-San Bernardino-Ontario, CA MSA   
3   METRO41860MM7360               San Francisco, CA HUD Metro FMR Area   
4   METRO47900M47900  Washington-Arlington-Alexandria, DC-VA-MD HUD ...   
..               ...                                                ...   
70  METRO42660MM7600            Seattle-Bellevue, WA HUD Metro FMR Area   
71  METRO47900M47900  Washington-Arlington-Alexandria, DC-VA-MD HUD ...   
72  METRO47900M47900  Washington-Arlington-Alexandria, DC-VA-MD HUD ...   
73  METRO47900M47900  Washington-Arlington-Alexandria, DC-VA-MD HUD ...   
74  METRO47900M47900  Washington-Arlington-Alexandria, DC-VA-MD HUD ...   

    il_fiscal_year  median_income  l50_1  l50_2  l50_3  l50_4  l50_5  l50_6  \
0             2015  

In [36]:
stacked_il_df['OMB13CBSA'].dtype

dtype('O')

#### Merging the AHS and IL datasets so new `'AMI'` variable can be computed and renter households can be catagorized by AMI based on the household income (`'HINCP'`).

In [37]:
#Merging AHS and IL DataFrames
metro_ami_panel_df = pd.merge(
    ahs_panel_df,
    stacked_il_df,
    left_on=['OMB13CBSA', 'SRVYEAR'],
    right_on=['OMB13CBSA', 'il_fiscal_year'],
    how='left',  #use left since I only want records from AHS
    indicator=True
)

### Validating that the join worked correctly
1. Checking `_merge` column to see whether each row came from the left, right, or both DataFrames. All or most values should be `both`.
2. Check for `NaN` values in a colum that should always be present if the merge worked. A result near `0.0` means that all values are present.
3. Cross-tabulate year values to spot mismatches
4. Inspect unmatched rows - look at examples of merge failures

In [38]:
#Checking `_merge` column
metro_ami_panel_df['_merge'].value_counts()

_merge
both          173009
left_only     142470
right_only         0
Name: count, dtype: int64

This equals 315,479, same length as the `ahs_panel_df`.

In [39]:
#Checking `NaN` values
metro_ami_panel_df['median_income'].isna().mean()

0.4515989970806298

In [40]:
#Cross-tab year values to spot mismatches
pd.crosstab(metro_ami_panel_df['SRVYEAR'], metro_ami_panel_df['_merge'])

_merge,left_only,both
SRVYEAR,,
2015,31135,37415
2017,29547,36434
2019,27281,35164
2021,29137,34297
2023,25370,29699


In [41]:
#Inspecting some of the unmatched rows
metro_ami_panel_df[metro_ami_panel_df['_merge'] != 'both'].head(25)

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,...,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,_merge
0,11000002,1600,-6,-6,8,99998,813.89,03,-6,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,11000003,840,2,-6,8,99998,581.10,03,2023,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,11000005,-6,1,-6,-6,99998,7335.97,02,1995,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,11000006,-6,1,-6,-6,99998,6562.87,02,2019,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,11000008,800,2,-6,8,99998,1490.80,06,2019,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,11000009,-6,-6,-6,-6,99998,2210.13,02,-6,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,11000010,1400,2,-6,1,99998,234.19,02,2020,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,11000019,1200,2,-6,1,99998,710.49,09,2022,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11,11000021,3000,2,-6,8,99998,5647.69,04,2018,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,11000025,250,2,-6,1,99998,609.41,06,2018,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [42]:
#checking NaN values in 'OMB13CBSA' column
# Count NaN values for each selected column within each OMB13CBSA group
nan_counts_by_cbsa = (
    metro_ami_panel_df
    .groupby('OMB13CBSA')[[
        'hud_area_code', 'hud_area_name', 'il_fiscal_year', 'median_income',
        'l50_1', 'l50_2', 'l50_3', 'l50_4', 'l50_5', 'l50_6', 'l50_7', 'l50_8',
        'ELI_1', 'ELI_2', 'ELI_3', 'ELI_4', 'ELI_5', 'ELI_6', 'ELI_7', 'ELI_8',
        'l80_1', 'l80_2', 'l80_3', 'l80_4', 'l80_5', 'l80_6', 'l80_7', 'l80_8'
    ]]
    .apply(lambda df: df.isna().sum())
)

# View the result
with pd.option_context('display.max_columns', None):
    display(nan_counts_by_cbsa)  

,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,l50_7,l50_8,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8
OMB13CBSA,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12060,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16980,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19820,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26420,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35620,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Checking a random group of owner/renter-occupied housing units in the top 15 largest metropolitan areas to see if merge worked correctly

In [43]:
cbsa_values = [
    '12060', '14460', '16980', '19100', '19820',
    '26420', '31080', '33100', '35620', '37980',
    '38060', '40140', '41860', '42660', '47900'
]

# Filter your dataframe to include only the specified CBSAs
filtered_df = metro_ami_panel_df[metro_ami_panel_df['OMB13CBSA'].isin(cbsa_values)]

# Randomly sample 25 rows from the filtered dataframe
sample_df = filtered_df.sample(n=25, random_state=4)

# Display or work with the result
with pd.option_context('display.max_columns', None):
    display(sample_df)  

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,SAMEHHLD,YRBUILT,BEDROOMS,VACMONTHS,HUDSUB,VACANCY,HINCP,TOTHCAMT,SRVYEAR,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,l50_7,l50_8,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,_merge
29428,11053729,800,2,1,2,35620,576.98,09,2014,3,3,1950,3,-6,2,-6,49980,880,2023,METRO35620MM5600,"New York, NY HUD Metro FMR Area",2023.00,94400.00,49450.00,56500.00,63550.00,70600.00,76250.00,81900.00,87550.00,93200.00,29650.00,33900.00,38150.00,42350.00,45750.00,49150.00,52550.00,55950.00,79200.00,90500.00,101800.00,113100.00,122150.00,131200.00,140250.00,149300.00,both
106346,11079367,-6,-6,-6,-6,16980,1225.66,02,-6,-6,-6,1940,2,25,-6,07,-6,-6,2021,METRO16980M16980,"Chicago-Joliet-Naperville, IL HUD Metro FMR Area",2021.00,93200.00,32650.00,37300.00,41950.00,46600.00,50350.00,54100.00,57800.00,61550.00,19600.00,22400.00,25200.00,27950.00,31040.00,35580.00,40120.00,44660.00,52200.00,59650.00,67100.00,74550.00,80550.00,86500.00,92450.00,98450.00,both
268014,11026479,1200,-6,-6,8,33100,848.15,09,-6,-6,NaN,2000,2,2,-6,01,-6,-6,2015,METRO33100MM5000,"Miami-Miami Beach-Kendall, FL HUD Metro FMR Area",2015.00,49900.00,23700.00,27100.00,30500.00,33850.00,36600.00,39300.00,42000.00,44700.00,14250.00,16250.00,20090.00,24250.00,28410.00,32570.00,36730.00,40890.00,37950.00,43350.00,48750.00,54150.00,58500.00,62850.00,67150.00,71500.00,both
282537,11044598,2000,-6,2,8,31080,1486.76,07,-6,-6,NaN,1930,1,-6,-6,01,-6,-6,2015,METRO31080MM4480,"Los Angeles-Long Beach, CA HUD Metro FMR Area",2015.00,63000.00,29050.00,33200.00,37350.00,41500.00,44850.00,48150.00,51500.00,54800.00,17450.00,19950.00,22450.00,24900.00,28410.00,32570.00,36730.00,40890.00,46500.00,53150.00,59800.00,66400.00,71750.00,77050.00,82350.00,87650.00,both
48762,11088617,-6,1,-6,-6,40140,1071.99,02,2018,3,1,2018,4,-6,-6,-6,87000,3678,2023,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",2023.00,94500.00,32650.00,37300.00,41950.00,46600.00,50350.00,54100.00,57800.00,61550.00,19600.00,22400.00,25200.00,30000.00,35140.00,40280.00,45420.00,50560.00,52200.00,59650.00,67100.00,74550.00,80550.00,86500.00,92450.00,98450.00,both
247739,11001037,2700,2,2,8,41860,705.18,07,2012,1,NaN,2000,2,-6,3,-6,65000,2921,2015,METRO41860MM7360,"San Francisco, CA HUD Metro FMR Area",2015.00,101900.00,41050.00,46900.00,52750.00,58600.00,63300.00,68000.00,72700.00,77400.00,24650.00,28150.00,31650.00,35150.00,38000.00,40800.00,43600.00,46400.00,65700.00,75100.00,84500.00,93850.00,101400.00,108900.00,116400.00,123900.00,both
106608,11079774,2400,2,2,8,41860,791.67,05,2008,4,2,1940,2,-6,3,-6,126000,2590,2021,METRO41860MM7360,"San Francisco, CA HUD Metro FMR Area",2021.00,149600.00,63950.00,73100.00,82250.00,91350.00,98700.00,106000.00,113300.00,120600.00,38400.00,43850.00,49350.00,54800.00,59200.00,63600.00,68000.00,72350.00,102450.00,117100.00,131750.00,146350.00,158100.00,169800.00,181500.00,193200.00,both
157566,11056577,-6,1,-6,-6,42660,633.94,02,1977,1,4,1970,5,-6,-6,-6,128400,1188,2019,METRO42660MM7600,"Seattle-Bellevue, WA HUD Metro FMR Area",2019.00,108600.00,38750.00,44300.00,49850.00,55350.00,59800.00,64250.00,68650.00,73100.00,23250.00,26600.00,29900.00,33200.00,35900.00,38550.00,41200.00,43850.00,61800.00,70600.00,79450.00,88250.00,95350.00,102400.00,109450.00,116500.00,both
35349,11064142,3000,-6,2,8,35620,2920.11,09,-6,-6,-6,1960,1,-6,-6,02,-6,-6,2023,METRO35620MM5600,"New York, NY HUD Metro FMR Area",2023.00,94400.00,49450.00,56500.00,63550.00,70600.00,76250.00,81900.00,87550.00,93200.00,29650.00,33900.00,38150.00,42350.00,45750.00,49150.00,52550.00,55950.00,79200.00,90500.00,101800.00,113100.00,122150.00,131200.00,140250.00,149300.00,both
37851,11068792,1100,2,-6,8,40140,958.16,05,2019,2,1,1930,1,-6,3,-6,23600,1240,2023,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",2023.00,94500.00,32650.00,37300.00,41950.00,466

In [44]:
#Checking the shape/info of the new metro_ami_panel_df DataFrame
print(metro_ami_panel_df.shape)
print(metro_ami_panel_df.info())

(315479, 48)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315479 entries, 0 to 315478
Data columns (total 48 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   CONTROL         315479 non-null  object  
 1   RENT            315479 non-null  int64   
 2   TENURE          315479 non-null  object  
 3   RENTCNTRL       315479 non-null  object  
 4   RENTSUB         315479 non-null  object  
 5   OMB13CBSA       315479 non-null  object  
 6   WEIGHT          315479 non-null  float64 
 7   BLD             315479 non-null  object  
 8   HHMOVE          315479 non-null  int64   
 9   NUMPEOPLE       315479 non-null  int64   
 10  SAMEHHLD        246929 non-null  object  
 11  YRBUILT         315479 non-null  int64   
 12  BEDROOMS        315479 non-null  int64   
 13  VACMONTHS       315479 non-null  int64   
 14  HUDSUB          315479 non-null  object  
 15  VACANCY         315479 non-null  object  
 16  HINCP           315479 no

All obversations/rows were matched. Dropping the `'_merge'` column.

In [45]:
metro_ami_panel_df.drop(columns=['_merge'], inplace=True)

In [46]:
#Checking the DataFrame again
with pd.option_context('display.max_columns', None):
    display(metro_ami_panel_df.head())  

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,SAMEHHLD,YRBUILT,BEDROOMS,VACMONTHS,HUDSUB,VACANCY,HINCP,TOTHCAMT,SRVYEAR,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,l50_7,l50_8,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8
0,11000002,1600,-6,-6,8,99998,813.89,03,-6,-6,-6,1980,4,0,-6,02,-6,-6,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000003,840,2,-6,8,99998,581.10,03,2023,3,4,1970,2,-6,3,-6,48000,1093,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000005,-6,1,-6,-6,99998,7335.97,02,1995,2,1,1970,4,-6,-6,-6,292500,810,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000006,-6,1,-6,-6,99998,6562.87,02,2019,3,4,1970,3,-6,-6,-6,56000,489,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000008,800,2,-6,8,99998,1490.80,06,2019,1,4,1920,1,-6,3,-6,36000,845,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#Verify `CONTROL` is unique by survey year
grouped = metro_ami_panel_df.groupby('SRVYEAR')

for year, group in grouped:
    assert group['CONTROL'].nunique() == len(group), f"Control numbers are not unique for SRVYEAR {year}!"

In [48]:
#observing values of the `hud_area_name` column at a list
hud_area_name_list = metro_ami_panel_df['hud_area_name'].unique()

#observing unique CBSA codes
hud_area_code_list = metro_ami_panel_df['hud_area_code'].unique()
OMB13CBSA_list = metro_ami_panel_df['OMB13CBSA'].unique()

# Print the list
print(hud_area_name_list)
print(hud_area_code_list)
print(OMB13CBSA_list)

[nan 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA'
 'Washington-Arlington-Alexandria, DC-VA-MD HUD Metro FMR Area'
 'New York, NY HUD Metro FMR Area'
 'Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area'
 'San Francisco, CA HUD Metro FMR Area'
 'Houston-The Woodlands-Sugar Land, TX HUD Metro FMR Area'
 'Miami-Miami Beach-Kendall, FL HUD Metro FMR Area'
 'Atlanta-Sandy Springs-Roswell, GA HUD Metro FMR Area'
 'Phoenix-Mesa-Scottsdale, AZ MSA'
 'Chicago-Joliet-Naperville, IL HUD Metro FMR Area'
 'Dallas, TX HUD Metro FMR Area'
 'Detroit-Warren-Livonia, MI HUD Metro FMR Area'
 'Seattle-Bellevue, WA HUD Metro FMR Area'
 'Los Angeles-Long Beach-Glendale, CA HUD Metro FMR Area'
 'Riverside-San Bernardino-Ontario, CA MSA'
 'Houston-Baytown-Sugar Land, TX HUD Metro FMR Area'
 'Atlanta-Sandy Springs-Marietta, GA HUD Metro FMR Area'
 'Phoenix-Mesa-Glendale, AZ MSA'
 'Los Angeles-Long Beach, CA HUD Metro FMR Area']
[nan 'METRO37980M37980' 'METRO47900M47900' 'METRO35620MM5600'
 'METRO14460MM112

Since some Metro names change within the panel data, I will refer to the CBSA Codes (`'OMB13CBSA'`) for metro-based analyses.

## Categorizing Renter Households based on Area Median Income (AMI)/Income Limits.

I am creating a new `AMI` column consisting of catagorical variables (ELI, VLI, LI, and Above LI) that catagorizes each observation into an AMI group. I will include larger households by incresing the 8-person limit by 8% of the 4-person limit for each additional person beyond 8. 


### 1. Creating a function to assign an AMI catagory to renter households that have over eight (8) peoeple in the unit.

_This is from [HUD USER FY23 Income Limit Documentation](https://www.huduser.gov/portal/datasets/il//il23/IncomeLimitsMethodology-FY23.pdf)_

| **Number of Persons in Family and Percentage Adjustments** | 1    | 2    | 3    | 4    | 5     | 6     | 7     | 8     |
|:-----------------------------------------------------------:|:----:|:----:|:----:|:----:|:-----:|:-----:|:-----:|:-----:|
|                                                             | 70%  | 80%  | 90%  | Base | 108%  | 116%  | 124%  | 132%  |


HUD does not include income limits for families with more than eight persons in the printed lists because of space limitations. For each person over eight-persons, the four-person income limit should be multiplied by an additional 8 percent. (For example, the nine-person limit equals 140 percent \[132 + 8\] of the relevant four-person income limit.) HUD rounds income limits up to the nearest &#36;50.

**Note:**  HUD has used the very low-income limits as the basis for deriving other income limits, unless the relevant statutory language has no references or relationship to low- and very low-income limits, as defined by the U.S. Housing Act of 1937.

#### Example:

* Very Low-Inome limit for 4-person household = &#36;46,750 
* Very Low-Income limit for 8-person household = &#36;61,750 
    * (&#36;46,750 * .32) _rounded to the nearest &#36;50_  
* Low-Income limit for 9-person = &#36;65,450 
    * (&#36;46,750 * .40) _not rounded since the product is a multiple of &#36;50_

In [49]:
#Examinig how many renter/owner-occupied housing units have more than 8 people 
metro_ami_panel_df.groupby('TENURE')['NUMPEOPLE'].value_counts()

TENURE  NUMPEOPLE
-6      -6           41729
1        2           60226
         1           37296
         3           25044
         4           23769
         5           10332
         6            3831
         7            1329
         8             483
         9             200
         10             81
         11             36
         12             22
         13             10
         14              3
         15              2
         16              1
         18              1
2        1           45034
         2           28879
         3           15847
         4           11565
         5            5806
         6            2406
         7             955
         8             362
         9             135
         10             46
         11             24
         12             10
         13              6
         14              3
         15              2
         16              1
         17              1
         18              1
         1

356 owner-occupied housing units have more than 8 people in the unit. 230 renter-occupied housing units have more than 8 people in the unit.

In [50]:
def adjusted_thresholds(threshold_4_array, ppl_count_array):
    """
    Vectorized adjustment of income thresholds for households with more than 8 people.
    - Applies HUD rule: Add 8% of the 4-person threshold for each person beyond 8.
    - Returns NaN for households with 8 or fewer people (which will be handled in the function below)
    
    Parameters:
    - threshold_4_array: array of 4-person income thresholds
    - ppl_count_array: array of household sizes (number of people)
    
    Returns:
    - array of adjusted thresholds (or Nan if 'NUMPEOPLE' <= 8)
    """
    #Convert inputs to NumPy arrays if they are not already an array
    threshold_4_array = np.asarray(threshold_4_array)
    ppl_count_array = np.asarray(ppl_count_array)
    
    #Calculate how many obversations/renter households exceed 8 people
    extra_people = np.maximum(ppl_count_array - 8, 0)
    
    #Compute the total adjustment: 8% increase for each "extra" person and rounds to the nearest $50 
    adjustment = np.round((threshold_4_array * 0.08 * extra_people) / 50) * 50
    
    #Apply the adjusment only if there are extra people; else return NaN
    adjusted_thresholds = np.where(extra_people > 0, threshold_4_array + adjustment, np.nan)
    
    return adjusted_thresholds

### 2. Assigning the correct income threshold based on houhsehold size.

In [51]:
def get_threshold_by_size(df, base_col):
    """
    Return a pandas Series with the appropriate income threshold for each household size ('NUMPEOPLE' 1–8).
    
    Parameters:
    - df: DataFrame containing threshold columns like 'ELI_1' to 'ELI_8', 'l50_1' to l50_8`, etc.
    - base_cal: string, the prefix for the threshold columns ('ELI', 'l50', 'l80', etc.)
    
    Returns:
    - A pandas Series of thresholds matched to each household's NUMPEOPLE.
    """
    #Create list of column names to pull threshold values by size
    columns = [f'{base_col}_{i}' for i in range(1, 9)]
    
    #Convert these threshold columns to a NumPY matrix (2d array)
    matrix = df[columns].to_numpy()
    
    #Calculate the index of the column that corresponds to each row's NUMPEOPLE
    #Clip NUMPEOPLE to 1-8 range, subtract 1 for zero-based indexing
    indexer = df['NUMPEOPLE'].clip(1, 8).astype(int) - 1
    
    #Select the threshold from the appropriate column for each row
    return matrix[np.arange(len(df)), indexer]

### 3. Assign the AMI to each unique renter houhsehold
This is the final part of the function/method for assigning onwer/renter-occupied housing units to an **AMI categories** (Extremely Low Income `ELI`, Very Low Income `VLI`, Low Income `LI`, and Above Low Income `Above LI`) based on:
- AHS Household inomce variable (`'HINCP'`)
- AHS Household size (`'NUMPEOPLE'`)
- HUD Income Limts (`'ELI_1'` to `'ELI_8'`, `'l50_1'` to `'l50_8'`, etc.

In [52]:
def assign_ami_category(df):
    """
    Assign AMI category (ELI, VLI, LI, Above LI) to each unique renter household (`CONTROL`).
    
    Expects Columns:
    - 'HINCP': household income
    - 'NUMPEOPLE': household size
    - Thresholds: 'ELI_1' - 'ELI_8', 'l50_1' - 'l50_8', 'l80_1' - 'l80_8'
    - And 'ELI_4', 'l50_4', 'l80_4' for adjustments for households with more than 8 people
    """
    df = df.copy()
    
    
    #--------- Vectorized thresholds for ELI, VLI, LI ---------
    for prefix, out_col, fallback_col in [
        ('ELI', 'ELI_threshold', 'ELI_4'),
        ('l50', 'VLI_threshold', 'l50_4'),
        ('l80', 'LI_threshold', 'l80_4'),
    ]:
        #Threshold matrix for 1-8 people: shape (n_rows, 8)
        cols = [f'{prefix}_{i}' for i in range (1, 9)]
        matrix = df[cols].to_numpy()
        
        #Index into the threshold matrix using NUMPEOPLE
        ppl_idx = df['NUMPEOPLE'].clip(1, 8).astype(int) - 1
        size_thresholds = matrix[np.arange(len(df)), ppl_idx]
        
        #Adjust for larger households (>8)
        adjusted = adjusted_thresholds(df[fallback_col], df['NUMPEOPLE'])
        
        #Combine adjusted and base thresholds
        df[out_col] = np.where(df['NUMPEOPLE'] <= 8, size_thresholds, adjusted)
        
    #--------- AMI assignment (Fully vectorized) ---------    
    conditions = [
        df['HINCP'] <= df['ELI_threshold'],
        df['HINCP'] <= df['VLI_threshold'],
        df['HINCP'] <= df['LI_threshold']
    ]
    choices = ['ELI', 'VLI', 'LI']    
    df['AMI'] = np.select(conditions, choices, default='Above LI')
    
    # Set AMI = NaN if any of the key inputs is missing
    required_columns = [
        'l50_1', 'l50_2', 'l50_3', 'l50_4', 'l50_5', 'l50_6', 'l50_7', 'l50_8',
        'ELI_1', 'ELI_2', 'ELI_3', 'ELI_4', 'ELI_5', 'ELI_6', 'ELI_7', 'ELI_8',
        'l80_1', 'l80_2', 'l80_3', 'l80_4', 'l80_5', 'l80_6', 'l80_7', 'l80_8',
        'ELI_threshold', 'VLI_threshold', 'LI_threshold'
    ]
    invalid_mask = df[required_columns].isna().any(axis=1) | (df['HINCP'] == -6)
    df.loc[invalid_mask, 'AMI'] = np.nan

    return df

### 4. Using the functions to assign an `'AMI'` category and examining `metro_ami_panel_df` after the funciton was executed.

In [53]:
#Usage
metro_ami_panel_df = assign_ami_category(metro_ami_panel_df)

In [54]:
#Looking at the newly created DataFrame with the 'AMI' values
metro_ami_panel_df.head()

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,ELI_threshold,VLI_threshold,LI_threshold,AMI
0,11000002,1600,-6,-6,8,99998,813.89,03,-6,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000003,840,2,-6,8,99998,581.10,03,2023,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000005,-6,1,-6,-6,99998,7335.97,02,1995,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000006,-6,1,-6,-6,99998,6562.87,02,2019,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000008,800,2,-6,8,99998,1490.80,06,2019,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#Preliminary review of AMI counts over the survey years.
print(metro_ami_panel_df.groupby(['SRVYEAR', 'TENURE'])['AMI'].value_counts())

SRVYEAR  TENURE  AMI     
2015     1       Above LI    13242
                 LI           2587
                 ELI          2120
                 VLI          1601
         2       Above LI     5026
                 ELI          4258
                 LI           2324
                 VLI          2101
2017     1       Above LI    13064
                 LI           2587
                 ELI          2020
                 VLI          1607
         2       Above LI     5043
                 ELI          3684
                 LI           2377
                 VLI          1963
2019     1       Above LI    12435
                 LI           2618
                 ELI          2079
                 VLI          1608
         2       Above LI     4633
                 ELI          3444
                 LI           2259
                 VLI          1816
2021     1       Above LI    11384
                 LI           2515
                 ELI          2107
                 VLI         

In [56]:
#Looking at number of NaN values in the 'AMI' column
metro_ami_panel_df['AMI'].isna().sum()

162024

In [57]:
#checking NaN values in 'OMB13CBSA' column
# Count NaN values for each selected column within each OMB13CBSA group
nan_ami_counts_by_cbsa = (
    metro_ami_panel_df
    .groupby('OMB13CBSA')[['ELI_threshold', 'VLI_threshold', 'LI_threshold', 'AMI'
    ]]
    .apply(lambda df: df.isna().sum())
)

# View the result
with pd.option_context('display.max_columns', None):
    display(nan_ami_counts_by_cbsa)  

,ELI_threshold,VLI_threshold,LI_threshold,AMI
OMB13CBSA,,,,
12060,0,0,0,1239
14460,0,0,0,767
16980,0,0,0,1175
19100,0,0,0,1152
19820,0,0,0,1430
26420,0,0,0,1571
31080,0,0,0,840
33100,0,0,0,2757
35620,0,0,0,1208


162,024 is the sum of NaNs from the number of observations in `'OMB13CBSA'` column that equal `'99998'` and `'99999'` (142,470) _**plus**_ the sum of NaN values within the 15 largest metropolitan areas that are vacant `'TENURE' == '-6'` (19,554).

### 5. Data Integrity Check

In [58]:
# Data integrity check
def check_data_integrity(df):
    print("🔍 DATA INTEGRITY CHECK\n" + "-"*30)
    
    total_rows = len(df)
    unique_ids = df[['CONTROL', 'SRVYEAR']].drop_duplicates().shape[0]
    control_duplicates = df['CONTROL'].duplicated().sum()
    
    print(f"Total rows: {total_rows}")
    print(f"Unique CONTROL + SRVYEAR pairs: {unique_ids}")
    print(f"Duplicate CONTROL values (across years): {control_duplicates}")
    
    # Nulls in key columns
    missing_income = df['HINCP'].isna().sum()
    missing_size = df['NUMPEOPLE'].isna().sum()
    missing_ami = df['AMI'].isna().sum()
    
    print(f"Missing household income (HINCP): {missing_income}")
    print(f"Missing household size (NUMPEOPLE): {missing_size}")
    print(f"Missing AMI category assignments: {missing_ami}")
    
    # AMI distribution
    print("\n📊 AMI Category Distribution:")
    print(df['AMI'].value_counts(dropna=False).sort_index())
    
    print("\n✅ All checks complete.\n")

In [59]:
#Usage
check_data_integrity(metro_ami_panel_df)

🔍 DATA INTEGRITY CHECK
------------------------------
Total rows: 315479
Unique CONTROL + SRVYEAR pairs: 315479
Duplicate CONTROL values (across years): 223708
Missing household income (HINCP): 0
Missing household size (NUMPEOPLE): 0
Missing AMI category assignments: 162024

📊 AMI Category Distribution:
AMI
Above LI     82514
ELI          30043
LI           23617
VLI          17281
NaN         162024
Name: count, dtype: int64

✅ All checks complete.



In [60]:
#Looking at the instance were `NUMPEOPLE` is greater than 8 people
metro_ami_panel_df.loc[metro_ami_panel_df['NUMPEOPLE'] > 8]

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,...,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,ELI_threshold,VLI_threshold,LI_threshold,AMI
11,11000021,3000,2,-6,8,99998,5647.69,04,2018,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,11001286,1700,2,-6,8,99998,825.56,02,2017,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3642,11006591,50,2,2,6,41860,882.98,02,2007,10,...,133800.00,148650.00,160550.00,172450.00,184350.00,196250.00,64650.00,107750.00,172450.00,Above LI
4497,11008115,-6,1,-6,-6,99998,4039.95,02,2022,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4722,11008539,350,2,-6,8,99998,5936.33,02,2008,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312024,11081015,-6,1,-6,-6,19820,707.01,02,1996,10,...,48750.00,54150.00,58500.00,62850.00,67150.00,71500.00,28150.00,39250.00,62800.00,Above LI
313079,11082338,-6,1,-6,-6,99998,3877.67,02,1999,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313211,11082507,-6,1,-6,-6,33100,1138.75,02,2009,14,...,48750.00,54150.00,58500.00,62850.00,67150.00,71500.00,35900.00,50100.00,80150.00,ELI
313957,11083428,-6,1,-6,-6,99999,5185.82,02,2006,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Checking a random group of owner/renter-occupied housing units in the top 15 largest metropolitan areas to see if AMI assignment worked correctly

In [61]:
# Filter your dataframe to include only the specified CBSAs (defined in code line 49)
filtered_ami_df = metro_ami_panel_df[metro_ami_panel_df['OMB13CBSA'].isin(cbsa_values)]

# Randomly sample 25 rows from the filtered dataframe
sample_ami_df = filtered_ami_df.sample(n=25, random_state=12)

# Display or work with the result
with pd.option_context('display.max_columns', None):
    display(sample_ami_df)  

,CONTROL,RENT,TENURE,RENTCNTRL,RENTSUB,OMB13CBSA,WEIGHT,BLD,HHMOVE,NUMPEOPLE,SAMEHHLD,YRBUILT,BEDROOMS,VACMONTHS,HUDSUB,VACANCY,HINCP,TOTHCAMT,SRVYEAR,hud_area_code,hud_area_name,il_fiscal_year,median_income,l50_1,l50_2,l50_3,l50_4,l50_5,l50_6,l50_7,l50_8,ELI_1,ELI_2,ELI_3,ELI_4,ELI_5,ELI_6,ELI_7,ELI_8,l80_1,l80_2,l80_3,l80_4,l80_5,l80_6,l80_7,l80_8,ELI_threshold,VLI_threshold,LI_threshold,AMI
106217,11079177,1600,2,-6,5,35620,5575.21,08,2010,2,1,1980,1,-6,3,-6,61000,1670,2021,METRO35620MM5600,"New York, NY HUD Metro FMR Area",2021.00,81700.00,41800.00,47750.00,53700.00,59650.00,64450.00,69200.00,74000.00,78750.00,25100.00,28650.00,32250.00,35800.00,38700.00,41550.00,44400.00,47300.00,66850.00,76400.00,85950.00,95450.00,103100.00,110750.00,118400.00,126000.00,28650.00,47750.00,76400.00,LI
138301,11028821,-6,1,-6,-6,35620,5327.57,02,2007,3,1,1980,4,-6,-6,-6,155000,4184,2019,METRO35620MM5600,"New York, NY HUD Metro FMR Area",2019.00,75500.00,37350.00,42700.00,48050.00,53350.00,57650.00,61900.00,66200.00,70450.00,22400.00,25600.00,28800.00,32000.00,34600.00,37150.00,39700.00,43430.00,59750.00,68300.00,76850.00,85350.00,92200.00,99050.00,105850.00,112700.00,28800.00,48050.00,76850.00,Above LI
284394,11046932,-6,1,-6,-6,31080,1648.04,02,1988,2,NaN,1919,3,-6,-6,-6,44400,690,2015,METRO31080MM4480,"Los Angeles-Long Beach, CA HUD Metro FMR Area",2015.00,63000.00,29050.00,33200.00,37350.00,41500.00,44850.00,48150.00,51500.00,54800.00,17450.00,19950.00,22450.00,24900.00,28410.00,32570.00,36730.00,40890.00,46500.00,53150.00,59800.00,66400.00,71750.00,77050.00,82350.00,87650.00,19950.00,33200.00,53150.00,LI
112096,11088402,-6,1,-6,-6,19100,1579.56,02,2017,2,3,2016,3,-6,-6,-6,35100,417,2021,METRO19100M19100,"Dallas, TX HUD Metro FMR Area",2021.00,89000.00,31150.00,35600.00,40050.00,44500.00,48100.00,51650.00,55200.00,58750.00,18700.00,21400.00,24050.00,26700.00,31040.00,35580.00,40120.00,44660.00,49850.00,57000.00,64100.00,71200.00,76900.00,82600.00,88300.00,94000.00,21400.00,35600.00,57000.00,VLI
85592,11047945,1200,2,1,8,31080,2241.31,05,1987,1,1,1980,2,-6,3,-6,0,1280,2021,METRO31080MM4480,"Los Angeles-Long Beach-Glendale, CA HUD Metro ...",2021.00,80000.00,41400.00,47300.00,53200.00,59100.00,63850.00,68600.00,73300.00,78050.00,24850.00,28400.00,31950.00,35450.00,38300.00,41150.00,44000.00,46800.00,66250.00,75700.00,85150.00,94600.00,102200.00,109750.00,117350.00,124900.00,24850.00,41400.00,66250.00,ELI
153382,11050587,1200,2,-6,5,35620,3571.73,08,1991,2,1,1970,2,-6,3,-6,55820,1290,2019,METRO35620MM5600,"New York, NY HUD Metro FMR Area",2019.00,75500.00,37350.00,42700.00,48050.00,53350.00,57650.00,61900.00,66200.00,70450.00,22400.00,25600.00,28800.00,32000.00,34600.00,37150.00,39700.00,43430.00,59750.00,68300.00,76850.00,85350.00,92200.00,99050.00,105850.00,112700.00,25600.00,42700.00,68300.00,LI
70774,11024948,1300,-6,-6,-9,19100,1011.22,02,-6,-6,-6,2000,4,-9,-6,01,-6,-6,2021,METRO19100M19100,"Dallas, TX HUD Metro FMR Area",2021.00,89000.00,31150.00,35600.00,40050.00,44500.00,48100.00,51650.00,55200.00,58750.00,18700.00,21400.00,24050.00,26700.00,31040.00,35580.00,40120.00,44660.00,49850.00,57000.00,64100.00,71200.00,76900.00,82600.00,88300.00,94000.00,18700.00,31150.00,49850.00,NaN
231388,11066575,-6,1,-6,-6,40140,523.87,02,1997,2,3,1970,4,-6,-6,-6,163000,2014,2017,METRO40140M40140,"Riverside-San Bernardino-Ontario, CA MSA",2017.00,63200.00,22600.00,25800.00,29050.00,32250.00,34850.00,37450.00,40000.00,42600.00,13550.00,16240.00,20420.00,24600.00,28780.00,32960.00,37140.00,41320.00,36150.00,41300.00,46450.00,51600.00,55750.00,59900.00,64000.00,68150.00,16240.00,25800.00,41300.00,Above LI
197990,11022683,-6,1,-6,-6,12060,986.11,02,2015,6,4,1990,3,-6,-6,-6,155000,463,2017,METRO12060M12060,"Atlanta-Sandy Springs-Roswell, GA HUD Metro FM...",2017.00,69700.00,24400.00,27900.00,31400.00,34850.00,37650.00,40450.00,43250.00,46050.00,14650.00,16750.00,20420.00,24600.00,28780.00,32960.00,37140.00,41320.00,39050.00,44600.00,50200.00,55750.00,60250

Based on reviewing random samples of the data, It appears as though the AMI assignment function worked.

## Exporting `metro_ami_panel_df` to csv for analysis in subsequent Notebooks.

In [ ]:
#Export the DataFrame to a CSV file at the specified location without the index
metro_ami_panel_df.to_csv('data/ahs/metro_ami_panel_df.csv', index=False)